2.2. 数据预处理

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始， 而不是从那些准备好的张量格式数据开始。 在Python中常用的数据分析工具中，我们通常使用pandas软件包。 像庞大的Python生态系统中的许多其他扩展包一样，pandas可以与张量兼容。 本节我们将简要介绍使用pandas预处理原始数据，并将原始数据转换为张量格式的步骤。 后面的章节将介绍更多的数据预处理技术。

2.2.1. 读取数据集
举一个例子，我们首先创建一个人工数据集，并存储在CSV（逗号分隔值）文件 ./data/house_tiny.csv中。 以其他格式存储的数据也可以通过类似的方式进行处理。 下面我们将数据集按行写入CSV文件中

In [ ]:
%cd ~/Hwj/深度学习代码练习/chapter2
!pwd

In [30]:

import os

os.makedirs(os.path.join('.','data'),exist_ok = True)
data_file = os.path.join('.','data','house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [1]:
import pandas as pd
data = pd.read_csv(data_file)
print(data)

NameError: name 'data_file' is not defined

2.2.2. 处理缺失值
注意，“NaN”项代表缺失值。 为了处理缺失的数据，典型的方法包括插值法和删除法， 其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。 在这里，我们将考虑插值法。

通过位置索引iloc，我们将data分成inputs和outputs， 其中前者为data的前两列，而后者为data的最后一列。 对于inputs中缺少的数值，我们用同一列的均值替换“NaN”项。

In [ ]:
input1,input2,outputs = data.iloc[:,0:1],data.iloc[:,1],data.iloc[:,2]
input1 = input1.fillna(input1.mean())
print(input1)
input2 = pd.get_dummies(input2,dummy_na = True)
print(input2)
#拼接dataFrame
inputs = pd.concat([input1,input2],axis = 1)
inputs


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。(现在好像改成了True和False)

In [70]:
import os
import pandas as pd
import numpy as np
import torch

os.makedirs(os.path.join('.','data'),exist_ok = True)
data_file_test = os.path.join('.','data','house_test.csv')

# 将 DataFrame 写入到 CSV 文件，不包含行索引
# inputs.to_csv('processed_inputs.csv', index=Fa

with open(data_file_test,'w') as ft:
    inputs.to_csv('./data/house_test.csv',index = False)


In [110]:
import os
import pandas as pd
import numpy as np
import torch

os.makedirs(os.path.join('.','data'),exist_ok = True)
data_file_test = os.path.join('.','data','house_test.csv')

data_t = pd.read_csv(data_file_test)
data_t

,NumRooms,Alley,Ssd,Price
0,NaN,Pave,shanke,127500
1,2.0,NaN,NaN,106000
2,4.0,NaN,NaN,178100
3,NaN,NaN,NaN,140000


In [111]:
input1,input2,outputs = data_t['NumRooms'],data_t[['Alley','Ssd']],data_t['Price']  
input1,input2,outputs

(0    NaN
 1    2.0
 2    4.0
 3    NaN
 Name: NumRooms, dtype: float64,
   Alley     Ssd
 0  Pave  shanke
 1   NaN     NaN
 2   NaN     NaN
 3   NaN     NaN,
 0    127500
 1    106000
 2    178100
 3    140000
 Name: Price, dtype: int64)

In [112]:
input1 = input1.fillna(input1.mean())
input1


0    3.0
1    2.0
2    4.0
3    3.0
Name: NumRooms, dtype: float64

In [113]:
input2 = pd.get_dummies(input2,dummy_na = True)
input2

,Alley_Pave,Alley_nan,Ssd_shanke,Ssd_nan
0,True,False,True,False
1,False,True,False,True
2,False,True,False,True
3,False,True,False,True


In [114]:
inputs = pd.concat([input1,input2,outputs],axis = 1)
inputs.to_csv('/home/extend2/user7/Hwj/深度学习代码练习/chapter2/data/house_test.csv',index = False)

In [141]:
import os 
import pandas as pd
import numpy as np

data_path = './data/house_test2.csv'
data = pd.read_csv(data_path)
data_output = './data/house_test2_output.csv'

input1,input2,ouputs = data.iloc[:,0:2],data.iloc[:,2:4],data.iloc[:,4:]
input1 = input1.fillna(input1.mean())
input2 = pd.get_dummies(input2,dummy_na = True)

inputs = pd.concat([input1,input2,outputs],axis = 1)
inputs.to_csv(data_output,index = False)
inputs


,NumRooms,Space,Alley_Pave,Alley_nan,Ssd_shanke,Ssd_nan,Price
0,3.0,100.0,True,False,True,False,127500
1,2.0,150.0,False,True,False,True,106000
2,4.0,200.0,False,True,False,True,178100
3,3.0,150.0,False,True,False,True,140000


2.2.3. 转换为张量格式
现在inputs和outputs中的所有条目都是数值类型，它们可以转换为张量格式。 当数据采用张量格式后，可以通过在 2.1节中引入的那些张量函数来进一步操作。

In [143]:
X = torch.tensor(inputs.to_numpy(dtype = np.float32))



AttributeError: 'Tensor' object has no attribute 'to_csv'

2.2.4. 小结
pandas软件包是Python中常用的数据分析工具中，pandas可以与张量兼容。
用pandas处理缺失的数据时，我们可根据情况选择用插值法和删除法。

练习：删除Nan最多的一列

In [145]:
data = pd.read_csv(data_path)
data

,NumRooms,Space,Alley,Ssd,Price
0,NaN,100.0,Pave,shanke,127500
1,2.0,NaN,NaN,shanke,106000
2,4.0,200.0,NaN,NaN,178100
3,NaN,NaN,NaN,NaN,140000


In [165]:
rows,columns = data.shape

max_columns = max([len([j for j in data.iloc[:,i] if pd.isna(j)]) for i in range(columns)])
max_index = [len([j for j in data.iloc[:,i] if pd.isna(j)]) for i in range(columns)].index(max_columns)
data.drop(max_index)
data

,NumRooms,Space,Alley,Ssd,Price
0,NaN,100.0,Pave,shanke,127500
1,2.0,NaN,NaN,shanke,106000
2,4.0,200.0,NaN,NaN,178100
3,NaN,NaN,NaN,NaN,140000
